## The content of this notebook:
 1_feature selection using mutal information  for features.



 2_using the dataset with the selected features to hyparameters tuning using optuna.




3_find the best hyperparamters after 10 trial and using the nested cross validation.




4_find the best model performance based on precision (we cand change the score matrix).





5_retrain the best model using the best parameters which we found.








In [ ]:
# @title ##import the necessary libraries M


# Libraries:

# Maths, data analysis, and graphics
import seaborn as sn
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import pickle

# Machine learning: models for training, and performance measures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

# Machine learning: models for training, and performance measures
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

##The preprocessing step for the data to find the beat features.


change the datasetname to your dataset name and extract the target classes.
The dataset we used is 'Hepatitis_C.csv' you can find here:
https://www.kaggle.com/datasets/fedesoriano/hepatitis-c-dataset

In [ ]:
df = pd.read_csv('Hepatitis_C.csv', sep=",")

In [ ]:
df

,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,label
0,32,0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7,0
1,45,0,41.7,73.2,43.6,29.4,6.4,8.89,5.31,71.0,67.4,70.3,0
2,55,0,41.5,59.5,15.4,16.2,6.8,6.35,5.22,80.0,12.4,69.9,0
3,53,0,37.8,98.1,30.5,21.1,4.0,5.02,4.42,94.0,23.2,65.2,0
4,56,1,39.7,66.0,14.2,20.8,3.5,7.48,5.88,66.0,7.2,67.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,62,1,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5,1
200,64,1,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3,1
201,64,1,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0,1
202,46,1,33.0,62.7,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0,1


In [ ]:

df = shuffle(df)
X_variant = df.drop(df.columns[[0]], axis=1)
y = df['label']

In [ ]:
X_variant

,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,label
120,0,46.3,71.4,30.3,22.5,26.9,7.95,7.30,88.0,21.8,71.8,0
60,0,41.0,63.5,16.9,21.6,5.9,6.03,4.74,83.0,13.5,73.7,0
2,0,41.5,59.5,15.4,16.2,6.8,6.35,5.22,80.0,12.4,69.9,0
45,1,43.7,61.3,18.0,23.3,4.3,9.57,6.04,70.0,18.6,75.2,0
160,0,45.0,26.9,23.1,125.0,17.0,6.97,4.01,60.5,72.2,73.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
90,0,42.9,92.6,21.6,26.1,7.4,12.86,5.73,94.0,42.9,70.1,0
162,0,39.0,62.7,118.0,62.0,10.0,7.28,3.50,72.0,74.0,81.0,1
98,1,41.2,38.2,18.6,20.5,9.3,6.15,5.44,64.0,9.7,70.9,0
80,1,44.5,53.4,13.4,17.5,5.4,7.74,5.55,66.0,15.4,71.9,0


## mutual information for features selection

new technique for feature selection ( which is mutual information for classification) and it is a Filter method calculates the mutual information between each feature in the input data (X_variant) and the target variable (y). Mutual information is a measure of the dependence or relationship between two random variables. In the context of feature selection, it helps assess how much information about the target variable is contained in each feature.

In [ ]:
#features selection
importance=mutual_info_classif(X_variant,y)


In [ ]:
feat_important=pd.Series(importance,X_variant.columns[0:len(X_variant.columns)] )

In [ ]:
feat_important = [feature for feature, importance in feat_important.items() if importance > 0.01]

In [ ]:
print(len(feat_important))
dataset_x=X_variant [feat_important]
dataset_x['target']=y

11


**the mutal information technique selected 41 features.** ,


 we still can explore other technique in selecting the best features after we find the beat model.

## exploring the dataset after features selection step:

In [ ]:
dataset_x.to_csv('mutual_information_results.csv', index=True)

In [ ]:
dataset_x.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

## Using the selected features of the dataset !!!
after finiding the best features we save the results into cv file to used independently


## Preprocessing the new dataset sperate the input features from the target

In [ ]:
dataset_x = pd.read_csv('mutual_information_results.csv', sep=",")

In [ ]:

dataset_x = shuffle(dataset_x)
x = dataset_x.drop(dataset_x.columns[[-1]], axis=1)
y = dataset_x["target"]

In [ ]:
y.reset_index(drop=True,inplace=True)


In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)


In [ ]:
x

array([[ 0.38207462, -1.13430292, -0.0133407 , ...,  0.73694538,
         0.75216196, -0.70710678],
       [ 0.11037711, -0.17730813, -0.88076405, ..., -0.43970725,
        -0.16374312, -0.70710678],
       [ 0.78962088,  1.38223893, -1.172886  , ..., -0.35155413,
        -0.00211281,  1.41421356],
       ...,
       [-0.89150745,  1.47084956, -0.30844349, ..., -0.51636215,
        -0.57679836, -0.70710678],
       [-1.01037511, -0.90391528,  0.028391  , ..., -0.41798837,
        -1.40290883, -0.70710678],
       [-0.63679104,  0.15941226,  0.44570808, ..., -0.25318034,
         0.84195658, -0.70710678]])

##Why nested cross validation ??
spliting the dataset between training and testing using sckitlearn has an issue that we may got lucky while choosing the testset and the smaples that we choose is easy to predict or it choose a test set full of outliers and make the predication is pessimistic..
using cross validation gave us the apportunity to experiments lots of possibilties which is great approach !! but what if we want to compare the performance of multiple classifiers ???!!
if we used the simple cross validation to compare bewteen the models we know that the performance of this model changes due to its configuration (hyperparameter).
here it come to play the role of using nested cross validation which is to loop of cross validation.
cross validation offers us a more accurate methode to estimate the performance of a given model while optimizing the hyperparamters at the same time
taking into the huge imbalance of our dataset the posibiltiy of the model to detect the most frequent target is more, and the possibility of building an optimistic model is higher.

In [ ]:
# @title Importing Libraries
import sklearn
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import matthews_corrcoef ,accuracy_score ,precision_score, recall_score, f1_score
from sklearn.metrics import matthews_corrcoef, accuracy_score, precision_score, recall_score, balanced_accuracy_score, \
    average_precision_score, fbeta_score, roc_auc_score, f1_score
import numpy as np
import sklearn
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef
from statistics import mean

In [ ]:
# @title installing the optuna Package
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.1 MB/s eta 0:00:00


In [ ]:
# @title ### Declare the inner and outer cross_validation

inner_cv =KFold(n_splits=2,shuffle=True,random_state=42)
outer_cv=KFold(n_splits=3,shuffle=True,random_state=42)


##Why using uptuna to search for best hyperparameters:
Manually tuning hyperparameters can be a tedious and time-consuming task. It often involves trial and error, as well as a deep understanding of the model and the data. Automated hyperparameter optimization tools, such as Optuna, offer a more efficient and systematic approach
###why not using the grid search !!??
grid search can be time-consuming and computationally expensive when the number of hyper-parameters or their possible values is large. Optuna, on the other hand, is a more advanced method that uses Bayesian optimization to efficiently search for the best set of hyper-parameters.
grid search : It searches the predetermined subset of the whole hyperparameter space of the target algorithm. while the uptuna is searches in very large sacle of hyperparamteres determined by range.



In [ ]:
# @title optunalancher function which find the best hyperparameters and the score of every trial


def objective(trial, classifier_name, x_train, y_train, x_val, y_val,clf):
    results_df=pd.DataFrame()
    # Invoke suggest methods of a trial object to generate hyperparameters
    if classifier_name == 'SVM':
        svr_c = trial.suggest_float('C', 1, 100)
        svr_gamma = trial.suggest_float('gamma', 0, 10)
        svr_kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf"])
        clf = SVC(C=svr_c, kernel=svr_kernel, gamma=svr_gamma,random_state=42)

    elif classifier_name == 'RandomForest':
        n_estimators = trial.suggest_int("n_estimators", 50, 1000, log=True)
        max_depth = trial.suggest_int("max_depth", 10, 100)
        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,random_state=42)

    elif classifier_name == "NaiveBayes":
        var_smoothing = trial.suggest_float("var_smoothing", 1e-10, 1)
        clf = GaussianNB(var_smoothing=var_smoothing)
    elif classifier_name == "XGB":
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
        max_depth = trial.suggest_int('max_depth', 3, 10, step=1)
        clf = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,random_state=42)
    elif classifier_name=='LR':
        max_iter=trial.suggest_int('max_iter',50,300)
        c_values =trial.suggest_int('C',10,100)
        solvers = trial.suggest_categorical('solver',['newton-cg', 'lbfgs', 'liblinear'])
        clf=LogisticRegression()

    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_val)
    Mcc = matthews_corrcoef(y_val, y_pred)
    Precision=precision_score(y_val, y_pred)
    F1= f1_score(y_val, y_pred,average='weighted')
    recall=recall_score(y_val, y_pred,average='weighted',zero_division=0)
    #res_df =get_outer_scores(y_val, y_pred, results_df, trial)
    #print(res_df)



    return   Precision


In [ ]:
dataset_x=pd.DataFrame(x)
y=pd.DataFrame(y)

In [ ]:
y

,target
0,0
1,0
2,1
3,0
4,0
...,...
199,0
200,0
201,0
202,0


In [ ]:
dataset_x

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.382075,-1.134303,-0.013341,0.732065,-0.220678,-0.332099,0.177552,0.619903,-0.168848,0.736945,0.752162,-0.707107
1,0.110377,-0.177308,-0.880764,0.008575,-0.465246,-0.210849,0.684787,-0.250643,0.204037,-0.439707,-0.163743,-0.707107
2,0.789621,1.382239,-1.172886,0.343770,-0.346907,-0.116544,0.380446,-1.014592,-0.054577,-0.351554,-0.002113,1.414214
3,-1.180186,-0.389974,-0.445562,-0.462689,-0.486942,-0.325363,-0.575498,-1.627528,-0.253047,-0.614736,-0.433127,-0.707107
4,-0.059434,-0.000087,-0.847975,-0.373083,-0.555973,-0.241162,-0.696454,0.202396,-0.265076,-0.580241,-0.343332,-0.707107
...,...,...,...,...,...,...,...,...,...,...,...,...
199,-0.008491,0.797409,0.651386,0.971016,-0.289709,-0.335467,1.121790,0.921929,0.011580,-0.214853,0.231353,-0.707107
200,-1.197167,-0.230475,0.216184,-0.525746,-0.621060,-0.005399,-0.392113,-0.090747,-0.180876,-0.541914,-1.456786,-0.707107
201,-0.891507,1.470850,-0.308443,-0.256926,-0.469191,-0.167065,0.243882,-0.463838,0.059695,-0.516362,-0.576798,-0.707107
202,-1.010375,-0.903915,0.028391,-0.532383,-0.719676,-0.359043,0.587242,1.481566,-0.241019,-0.417988,-1.402909,-0.707107


In [ ]:
# @title Define the classifiers Names in array:
classifiers = [ 'SVM','NaiveBayes','RandomForest','XGB','LR']


##optuna Trial and study :
#Trial manages all the single execution of model training, evaluation, and getting score by specifying one trial of the hyperparameter.

#Study manages and records all the entirety of Trials executed. This record helps us to know the best hyperparameter and suggests the next parameter space to search.

In [ ]:
# @title ##running the nested cross validation to find the best model and best hyperparameters
results = {}
best_param_list=[]
best_params = {}
mean_score_per_clf=[]
Mean_score_result=[]
results_of_clf_param=[]

for classifier_name in classifiers:
  study = optuna.create_study(direction='maximize',study_name=f'Hyperparameter-Tuning{classifier_name}')
  clf=SVC()
  best_score=[]
  #print(f'Running nested CV for {classifier_name}...')
  for train_index, test_index in outer_cv.split(dataset_x, y):
      x_train, x_test = dataset_x.iloc[train_index], dataset_x.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]



      for train_index_inner, test_index_inner in inner_cv.split(x_train, y_train):
          x_intrain, x_val = x_train.iloc[train_index_inner], x_train.iloc[test_index_inner]
          y_intrain, y_val = y_train.iloc[train_index_inner], y_train.iloc[test_index_inner]
          y_val=np.array(y_val)
          y_val=y_val.ravel()
          y_intrain=np.array(y_intrain)
          y_intrain=y_intrain.ravel()
          my_objective=lambda trial :objective(trial , classifier_name ,x_intrain,y_intrain ,x_val,y_val,clf)
          study.optimize(my_objective ,n_trials=10 , n_jobs=-1)
          best_params={f'{classifier_name}':study.best_params}
          best_param_list.append (best_params)

          best_score.append(study.best_value)
          #if fast_check:
            #break

      mean_score_per_clf.append(mean(best_score))


  results_of_clf_param.append(best_params)
  Mean_score_result.append({f'{classifier_name}': mean(mean_score_per_clf)})



[I 2024-02-06 16:57:11,791] A new study created in memory with name: Hyperparameter-TuningSVM
[I 2024-02-06 16:57:11,869] Trial 1 finished with value: 1.0 and parameters: {'C': 6.618184124008972, 'gamma': 7.874147772437382, 'kernel': 'poly'}. Best is trial 1 with value: 1.0.
[I 2024-02-06 16:57:11,899] Trial 0 finished with value: 1.0 and parameters: {'C': 54.358128266426995, 'gamma': 1.1503181324205192, 'kernel': 'rbf'}. Best is trial 1 with value: 1.0.
[I 2024-02-06 16:57:11,948] Trial 2 finished with value: 1.0 and parameters: {'C': 76.22383364021074, 'gamma': 8.520382077644763, 'kernel': 'poly'}. Best is trial 1 with value: 1.0.
[I 2024-02-06 16:57:12,016] Trial 3 finished with value: 1.0 and parameters: {'C': 71.32562314616017, 'gamma': 9.780793297212279, 'kernel': 'linear'}. Best is trial 1 with value: 1.0.
[I 2024-02-06 16:57:12,027] Trial 4 finished with value: 1.0 and parameters: {'C': 34.730293268073545, 'gamma': 8.94908803633017, 'kernel': 'poly'}. Best is trial 1 with value

In [ ]:
# @title Finding the best model over the rest models
def best_Model(Mean_score_result):
    max_entry = max(Mean_score_result, key=lambda entry: list(entry.values())[0])

    max_classifier = list(max_entry.keys())[0]
    max_value = list(max_entry.values())[0]

    print(f'The classifier with the maximum value is: {max_classifier}')
    print(f'The maximum value is: {max_value}')
    return max_classifier


In [ ]:
best_model=best_Model(Mean_score_result)

The classifier with the maximum value is: SVM
The maximum value is: 1.0


In [ ]:
best_model

'SVM'

In [ ]:
Mean_score_result

[{'SVM': 1.0},
 {'NaiveBayes': 1.0},
 {'RandomForest': 1.0},
 {'XGB': 1.0},
 {'LR': 1.0}]

In [ ]:
results_of_clf_param

[{'SVM': {'C': 6.618184124008972,
   'gamma': 7.874147772437382,
   'kernel': 'poly'}},
 {'NaiveBayes': {'var_smoothing': 0.5889680098015604}},
 {'RandomForest': {'n_estimators': 93, 'max_depth': 42}},
 {'XGB': {'n_estimators': 63,
   'learning_rate': 0.12411624302684149,
   'max_depth': 3}},
 {'LR': {'max_iter': 183, 'C': 26, 'solver': 'newton-cg'}}]

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold

In [ ]:
# @title Function used to train all my models in their best hyperparameters and find their scores


def Train_with_best_param(param, x, y):
    results_df = []
    Mcc=[]
    f1=[]
    pre=[]
    recall=[]
    x=pd.DataFrame(x)
    y=pd.DataFrame(y)
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    for result_dict in param:
            for clf, parameters in result_dict.items():

                for train_index, test_index in skf.split(x, y):
                    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
                    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
                    y_train_fold = np.array(y_train_fold).ravel()
                    y_test_fold = np.array(y_test_fold).ravel()


                    if clf == 'SVM':
                      model = SVC(**parameters)
                      idx = 1
                    elif clf == 'XGB':
                      model = XGBClassifier(**parameters)
                      idx = 2
                    elif clf == 'RandomForest':
                      model = RandomForestClassifier(**parameters)
                      idx = 3
                    elif clf == 'NaiveBayes':
                      model = GaussianNB(**parameters)
                      idx = 4
                    elif clf=='LR'   :
                      model=LogisticRegression(**parameters)

                    model.fit(x_train_fold, y_train_fold)
                    y_pred = model.predict(x_test_fold)
                    Mcc.append(matthews_corrcoef(y_test_fold, y_pred))
                    #results_df['MCC'] = Mcc
                    pre.append(precision_score(y_test_fold, y_pred,zero_division=0))
                    recall.append(recall_score(y_test_fold, y_pred))
                    f1.append(f1_score(y_test_fold, y_pred))
                    #results_df['Average_Precision'] = average_precision_score(y_test_fold, y_pred)
            results_df.append({"Model":clf,"MCC":mean(Mcc),"f1":mean(f1),"precision":mean(pre),"recall":mean(recall)})
    results_df=pd.DataFrame( results_df)

    return results_df


In [ ]:
res=Train_with_best_param(results_of_clf_param,dataset_x, y)

## scores matrices for the five classifiers in their best parameters

In [ ]:
res

,Model,MCC,f1,precision,recall
0,SVM,0.989252,0.992593,1.0,0.985714
1,NaiveBayes,0.914203,0.931610,1.0,0.883516
2,RandomForest,0.939219,0.951937,1.0,0.917582
3,XGB,0.951727,0.962101,1.0,0.934615
4,LR,0.961382,0.969681,1.0,0.947692


##Training the Final model in the best hyperparameters :

In [ ]:
# @title Save the best model with its hyperparameters in cv file
parameters=[]
for result_dict in results_of_clf_param:
      for clf, parameter in result_dict.items():
        if clf == best_model:
          parameters.append(parameter)
          best_param_df=pd.DataFrame(parameters)
          best_param_df["Model"]=best_model
best_param_df.to_csv ('best_model_with_best_hyperparameters.cvs',index=True)


In [ ]:
x=pd.DataFrame(x)
y=pd.DataFrame(y)

In [ ]:
def find_score(y_test_fold,y_pred):
      Mcc=0
      f1=0
      pre=0
      recall=0
      Mcc=matthews_corrcoef(y_test_fold, y_pred)
      #print(Mcc)
      pre=precision_score(y_test_fold, y_pred,zero_division=0)
      recall=recall_score(y_test_fold, y_pred)
      f1=f1_score(y_test_fold, y_pred)
      return Mcc,pre,recall,f1

In [ ]:
best_model_parameters = None

for model_params in results_of_clf_param:
    if best_model in model_params:
        best_model_parameters = model_params[best_model].items()
        break
print(f"Parameters of the best model ({best_model}):", best_model_parameters)
best_model_parameters

Parameters of the best model (SVM): dict_items([('C', 6.618184124008972), ('gamma', 7.874147772437382), ('kernel', 'poly')])


dict_items([('C', 6.618184124008972), ('gamma', 7.874147772437382), ('kernel', 'poly')])

In [ ]:
def train_best_model(x,y,results_of_clf_param):
  results_df = []
  skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
  Mcc=[]
  f1=[]
  pre=[]
  recall=[]
  for result_dict in results_of_clf_param:
      parameters = {}

      for model_params in results_of_clf_param:
          if best_model in model_params:
            parameters = model_params[best_model]

            if best_model == 'XGB':
              for train_index, test_index in skf.split(x, y):
                x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
                y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
                y_train_fold = np.array(y_train_fold).ravel()
                y_test_fold = np.array(y_test_fold).ravel()

                model = XGBClassifier(**parameters)
                model.fit(x_train_fold, y_train_fold)
                y_pred = model.predict(x_test_fold)
                MCC,p,recall_sc,F1=find_score(y_test_fold,y_pred)
                Mcc.append(MCC)
                pre.append(p)
                recall.append(recall_sc)
                f1.append(F1)

            elif best_model == 'SVM':
              for train_index, test_index in skf.split(x, y):
                x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
                y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
                y_train_fold = np.array(y_train_fold).ravel()
                y_test_fold = np.array(y_test_fold).ravel()

                model = SVC(**parameters, probability=True)
                model.fit(x_train_fold, y_train_fold)
                y_pred = model.predict(x_test_fold)
                MCC,p,recall_sc,F1=find_score(y_test_fold,y_pred)
                Mcc.append(MCC)
                pre.append(p)
                recall.append(recall_sc)
                f1.append(F1)

            elif best_model== 'LR':
              for train_index, test_index in skf.split(x, y):
                x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
                y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
                y_train_fold = np.array(y_train_fold).ravel()
                y_test_fold = np.array(y_test_fold).ravel()

                model = LogisticRegression(**parameters)
                model.fit(x_train_fold, y_train_fold)
                y_pred = model.predict(x_test_fold)
                MCC,p,recall_sc,F1=find_score(y_test_fold,y_pred)
                Mcc.append(MCC)
                pre.append(p)
                recall.append(recall_sc)
                f1.append(F1)
                break
            elif best_model == 'NaiveBayes':
              for train_index, test_index in skf.split(x, y):
                x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
                y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
                y_train_fold = np.array(y_train_fold).ravel()
                y_test_fold = np.array(y_test_fold).ravel()

                model = GaussianNB(**parameters)
                model.fit(x_train_fold, y_train_fold)
                y_pred = model.predict(x_test_fold)
                MCC,p,recall_sc,F1=find_score(y_test_fold,y_pred)
                Mcc.append(MCC)
                pre.append(p)
                recall.append(recall_sc)
                f1.append(F1)

            elif clf == 'RandomForest':
              for train_index, test_index in skf.split(x, y):
                x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
                y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
                y_train_fold = np.array(y_train_fold).ravel()
                y_test_fold = np.array(y_test_fold).ravel()

                model = RandomForestClassifier(**parameters)
                model.fit(x_train_fold, y_train_fold)
                y_pred = model.predict(x_test_fold)
                MCC,p,recall_sc,F1=find_score(y_test_fold,y_pred)
                Mcc.append(MCC)
                pre.append(p)
                recall.append(recall_sc)
                f1.append(F1)



      results_df.append({"Model":best_model,"MCC":mean(Mcc),"f1":mean(f1),"precision":mean(pre),"recall":mean(recall)})
      results_df=pd.DataFrame( results_df)

      return results_df, model

##Best model scores and its best hyperparameters

In [ ]:
train_best_model(x,y,results_of_clf_param)

(  Model       MCC        f1  precision    recall
 0   SVM  0.989252  0.992593        1.0  0.985714,
 SVC(C=6.618184124008972, gamma=7.874147772437382, kernel='poly',
     probability=True))